In [1]:
import ipywidgets as ipw
import bqplot as bq
import numpy as np
from bqplot.marks import Graph
from collections import defaultdict
import networkx as nx

In [2]:
def rand_dag(n):
    "Generate random DAG adjacency matrix."
    return np.tril(np.random.randint(0, 2, [n, n]), k=-1)

In [3]:
def gen_bq_dag(adj_mat):
    "Generate bqplot Graph object of DAG given adjacency matrix."
    
    nxg = nx.from_numpy_matrix(adj_mat)

    pos = nx.nx_pydot.graphviz_layout(nxg, prog='dot')
    x, y = np.array([pos[i] for i in range(N)]).T

    link_data = [{'source': source, 'target': target} for source, target in nxg.edges()]

    graph = Graph(
        node_data=node_data,
        link_data=link_data,
        scales=scales,
        link_type='line',
        highlight_links=False,
        x=x, y=y
    )
    
    return nxg, graph

In [4]:
fig_layout = ipw.Layout(width='600px', height='800px')

In [5]:
xs = bq.LinearScale()
ys = bq.LinearScale()
scales = {'x': xs, 'y': ys}

In [6]:
node_data = list('ABCDEF')
N = len(node_data)

In [7]:
adj_mat = rand_dag(N)
nxg, graph = gen_bq_dag(adj_mat)
f = bq.Figure(marks=[graph], layout=fig_layout)
f

A Jupyter Widget

In [8]:
f.marks[0].link_data

[{'source': 0, 'target': 5},
 {'source': 1, 'target': 4},
 {'source': 1, 'target': 5},
 {'source': 2, 'target': 3},
 {'source': 2, 'target': 4},
 {'source': 2, 'target': 5},
 {'source': 3, 'target': 5},
 {'source': 4, 'target': 5}]

In [9]:
nxg

In [10]:
graph.metadata = []

## Generate toy metadata

In [11]:
def gen_metadata(i):
    words = "cheese tacos alfredo avocado sasquatch alfred omega landlady".split()
    return dict(
        name='Step {}'.format(graph.node_data[i]),
        date='Tuesday the {}th'.format(30-i),
        word=words[i]
    )

In [12]:
for i in range(len(graph.node_data)):
    graph.metadata.append(gen_metadata(i))

In [13]:
graph.metadata

[{'date': 'Tuesday the 30th', 'name': 'Step A', 'word': 'cheese'},
 {'date': 'Tuesday the 29th', 'name': 'Step B', 'word': 'tacos'},
 {'date': 'Tuesday the 28th', 'name': 'Step C', 'word': 'alfredo'},
 {'date': 'Tuesday the 27th', 'name': 'Step D', 'word': 'avocado'},
 {'date': 'Tuesday the 26th', 'name': 'Step E', 'word': 'sasquatch'},
 {'date': 'Tuesday the 25th', 'name': 'Step F', 'word': 'alfred'}]

# Workflow Widget

In [14]:
class EditHTML(ipw.VBox):
    def __init__(self, value='', text_height=400):
        super().__init__()
        self.HTML = ipw.HTMLMath(value=value)
        self.Text = ipw.Textarea(value=value)
        self.ToggleButton = ipw.Button(description='Toggle')
        
        self.elements = [self.HTML, self.Text]
        self.descriptions = ['Edit', 'Render']
        ipw.jslink((self.HTML, 'value'), (self.Text, 'value'))
        
        # Set height and width of Textarea
        self.Text.layout.height = u'{}px'.format(text_height)
        self.Text.layout.width = u'95%'
        
        # Set HTML view by default
        self.set_view(0)
        
        self.ToggleButton.on_click(self.toggle)
    
    def set_view(self, state):
        self.state = state
        self.children = [self.elements[state], self.ToggleButton]
        self.ToggleButton.description = self.descriptions[state]
        
    def toggle(self, caller):
        self.set_view((self.state+1)%2)

In [15]:
class WorkflowWidget(ipw.HBox):
    "Widget to draw DAG via bqplot and provide node-level info/interaction."
    def __init__(self, bqgraph):
        super(WorkflowWidget, self).__init__()
        
        # Define variables
        self.bqgraph = bqgraph
        self._fig_layout = ipw.Layout(width='400px', height='600px')
        self._xs = bq.LinearScale()
        self._ys = bq.LinearScale()
        self._scales = {'x': xs, 'y': ys}
        mgin = 10
        
        # Define elements
        self._metadata_template = """
        Node name: {name}
        <br>
        Last modified: {date}
        <br>
        Description: {word}
        """
        self._metadata = ipw.HTML()
        
        readme_html = EditHTML(r"""
            <h1>Radiative Transfer</h1>

            The Radiative Transfer Equation is given by

            <p>
            $$\nabla I \cdot \omega = -c\, I(x, \omega) + \int_\Omega \beta(|\omega-\omega'|)\, I(x, \omega')$$
            </p>

            It is useful for
            <ul>
            <li>
            Stellar astrophysics
            </li>
            <li>
            Kelp
            </li>
            <li>
            Nice conversations
            </li>
            </ul>

            And is explained well by the following diagram.
            <br />
            <br />
            <img width=300px src="http://soap.siteturbine.com/uploaded_files/www.oceanopticsbook.info/images/WebBook/0dd27b964e95146d0af2052b67c7b5df.png" />
        """)
        self._notebook_button = ipw.Button(
            description='Open Notebook',
            button_style='success'
        )
        self._log_path_input = ipw.Text(
            description='Log path',
            value='/etc/login.defs'
        )
        self._log_html = ipw.HTML()
        
        self._readme_area = ipw.VBox([
            readme_html
        ])
        self._info_area = ipw.VBox([
            self._notebook_button,
            self._metadata
        ])
        self._log_area = ipw.VBox([
            self._log_path_input,
            self._log_html
        ])
        
        self._graph_plot = bq.Figure(
            marks=[self.bqgraph],
            layout=self._fig_layout
        )
        self._tab = ipw.Tab([
            self._readme_area,
            self._info_area,
            self._log_area
        ])
        
        # Define layout
        self.children = [
            self._graph_plot,
            self._tab,
        ]
        
        # Set attributes
        self._tab.set_title(0, 'Readme')
        self._tab.set_title(1, 'Info')
        self._tab.set_title(2, 'Log')
        self._tab.layout.height = self._fig_layout.height
        self._tab.layout.width = self._fig_layout.width
        
        #self._graph_plot.layout.border = '3px red solid'
        self._graph_plot.fig_margin = dict(
            left=mgin,
            right=mgin,
            bottom=mgin,
            top=mgin
        )
        self._graph_plot.min_aspect_ratio = 0
        
        # Graph style
        self.bqgraph.selected_style = dict(
            stroke='red'
        )
        
        # Default selections
        self._tab.selected_index = 0
        self.bqgraph.selected = [0]
        
        # Logic
        self.bqgraph.observe(self._call_update_metadata, names='selected')
        self._log_path_input.on_submit(self._call_read_log)
        
        # Run updates
        self._call_read_log()
        self._update_metadata(self.bqgraph.metadata[0])
    
    def _update_metadata(self, metadata):
        self._metadata.value = self._metadata_template.format(**metadata)
       
    def _call_update_metadata(self, change):
        try:
            new_name = self.bqgraph.metadata[change['new'][0]]
        except TypeError:
            new_name = defaultdict(str)
        self._update_metadata(new_name)
    
    def _read_log(self, log_path):
        try:
            with open(log_path) as log_file:
                log_text = log_file.read()
        except IOError:
            log_text = 'Error opening {}'.format(log_path)
        
        self._log_html.value = log_text
    
    def _call_read_log(self, caller=None):
        log_path = self._log_path_input.value
        self._read_log(log_path)

In [19]:
d = defaultdict(str)

In [24]:
d.__getitem__('name')

''

In [27]:
"'{name}'".format(**d)

"''"

In [28]:
w = WorkflowWidget(graph)
w

A Jupyter Widget

KeyError: 'name'

In [82]:
e = EditHTML(r"""
            <h1>Radiative Transfer</h1>

            The Radiative Transfer Equation is given by

            <p>
            $$\nabla I \cdot \omega = -c\, I(x, \omega) + \int_\Omega \beta(|\omega-\omega'|)\, I(x, \omega')$$
            </p>

            It is useful for
            <ul>
            <li>
            Stellar astrophysics
            </li>
            <li>
            Kelp
            </li>
            <li>
            Nice conversations
            </li>
            </ul>

            And is explained well by the following diagram.
            <br />
            <br />
            <img width=300px src="http://soap.siteturbine.com/uploaded_files/www.oceanopticsbook.info/images/WebBook/0dd27b964e95146d0af2052b67c7b5df.png" />
        """)
e

A Jupyter Widget

In [83]:
h = e.children[0]

In [63]:
t = e.children[0]

In [64]:
t.layout.height = u'300px'

In [62]:
h.layout.min_width

In [47]:
e.children

(HTML(value=''), Button(description='Toggle', style=ButtonStyle()))

In [24]:
ipw.jslink?

In [ ]:
!0

In [30]:
h = ipw.HTML()
t = ipw.Textarea()

In [31]:
ipw.jslink((h, 'value'), (t, 'value'))

In [33]:
o = ipw.Output()

In [36]:
b = ipw.VBox()

In [ ]:
t.on_

In [37]:
b.on_msg?

In [35]:
h.traits()

{'_display_callbacks': <traitlets.traitlets.Instance at 0x2afb9fb03160>,
 '_dom_classes': <traitlets.traitlets.Tuple at 0x2afb9fb03588>,
 '_model_module': <traitlets.traitlets.Unicode at 0x2afb9fb0bda0>,
 '_model_module_version': <traitlets.traitlets.Unicode at 0x2afb9fb0be48>,
 '_model_name': <traitlets.traitlets.Unicode at 0x2afb9fb5b3c8>,
 '_msg_callbacks': <traitlets.traitlets.Instance at 0x2afb9fb03198>,
 '_property_lock': <traitlets.traitlets.Dict at 0x2afb9fb030f0>,
 '_states_to_send': <traitlets.traitlets.Set at 0x2afb9fb03128>,
 '_view_count': <traitlets.traitlets.Int at 0x2afb9faf8f60>,
 '_view_module': <traitlets.traitlets.Unicode at 0x2afb9fb0be80>,
 '_view_module_version': <traitlets.traitlets.Unicode at 0x2afb9fb0beb8>,
 '_view_name': <traitlets.traitlets.Unicode at 0x2afb9fb5b390>,
 'comm': <traitlets.traitlets.Instance at 0x2afb9faf8f98>,
 'description': <traitlets.traitlets.Unicode at 0x2afb9fb1b860>,
 'keys': <traitlets.traitlets.List at 0x2afb9fb03048>,
 'layout': <i

In [ ]:
def toggle():
    

In [19]:
h

A Jupyter Widget

In [20]:
t

A Jupyter Widget